# Principal Component Analysis (PCA)


PCA is a technique for reducing the dimension of a $n \times p$ data matrix $X$ by singling out the directions under which $X$ varies the most. The principal components $Z_i$ may be written as a sum $Z_i = \sum_{j=1}^p \phi_{ij} X_j \ $  where $p$ is the number of original predictor variables $X_j$. The coefficients  $\phi_{ij}$ are determined by maximizing the variance under the constraint $\sum_{j}\phi_{ij}^2 = 1$ for all $i$. The first principal component vector has a special geometric interpretation because it defines the line that is as close as possible to the data. In other words, the first principal component line minimizes the sum of the squared perpendicular distances between each point and the line. In this notebook we use scikit-learn to obtain a PCA decomposition of the input matrix for housing price data and construct a linear regression model using the PCs as input to predict the price.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
data = pd.read_csv('Data/DATA_Housing_Prices.csv') # load housing data

In [ ]:
# Let's construct a dataframe with these features
data['Antiquity'] = data['YrSold'] - data['YearRemodAdd']

X = data.drop(columns=['Id','SalePrice','YearBuilt', 'YearRemodAdd', 'YrSold',
                       'LotFrontage','MasVnrArea','GarageYrBlt'], axis = 1)
y = np.log(data['SalePrice'])

data.describe() # inspect the data

In [ ]:
X = X.select_dtypes(include=np.number)
X_cols = X.columns
X.head()

In [ ]:
X.isna().sum() # check for missing values

# Scaling the data before PCA

In [ ]:
# split the data into test/train sets for machine learning
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

scaler = StandardScaler() # transform to a new variable z=(x-mu)/sigma, which scales and shifts the data

scaler.fit(X_train) # get the parameters mu and sigma for each column (from the train set!)

X_train = scaler.transform(X_train) # transform
X_test  = scaler.transform(X_test) # transform

X_train = pd.DataFrame(X_train, columns = X.columns) # repack
X_test  = pd.DataFrame(X_test , columns = X.columns)

In [ ]:
X_train.shape

In [ ]:
X_train

# Performing PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(svd_solver='full') # singular value decomp

pca.fit(X_train) # fit to train

X_train = pca.transform(X_train) # transform
X_test  = pca.transform(X_test)  # transform

cols = ['PCA_' + str(i) for i in list(range(1,len(X.columns)+1))] # rename columns

X_train = pd.DataFrame(X_train, columns = cols)
X_test  = pd.DataFrame(X_test , columns = cols)

In [ ]:
# select the top 15 PCs
X_train = X_train.iloc[:,:15]
X_test = X_test.iloc[:,:15]
X_train

# Getting variances

In [ ]:
print(pca.explained_variance_ratio_) # Individual variances of each principal components
print(pca.singular_values_)

In [ ]:
variances = pca.explained_variance_ratio_.tolist()
variances[:5]

In [ ]:
total_variance = [sum(variances[:i]) for i in range(1, len(variances)+1)]
scree = pd.DataFrame({'num_pca': range(1,len(variances)+1),'variance': variances,'cumulative_variance': total_variance})
scree.head(10)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style("darkgrid", {"axes.facecolor": ".9"})

fig, ax = plt.subplots(1,2,figsize=(16,8))

sns.lineplot(x="num_pca", y="variance", data=scree, ax = ax[0], color = 'black')
ax[0].set_xticks(range(1,32))
sns.barplot(x="num_pca", y="cumulative_variance", data=scree, ax = ax[1], color = 'lightblue')

Here we have a very bad scenario. On the left plot it looks that there is an elbow at 6 pca's. However, with five principal components we are only able to explain only 51% of the total variability of the data!!! This can be seen from

```
cumulative_variance[6] = 0.51
```
Bear in mind that we only selected the numerical columns and we didn't take into account the categorical ones. 


# pca coefficients

In [ ]:
pca.components_

In [ ]:
# First pca components. The first principal component direction of the data is that along which the observations vary the most.
pca.components_[0,:]

In [ ]:
# decomposition of the PCAs in terms of the original variables
for comp in range(1,len(pca.components_)+1):
    print("Principal component ",comp)
    my_list = [(abs(value),index+1,X.columns[index],value) for index,value in enumerate(pca.components_[comp-1,:]) ]
    my_list.sort(reverse=True)
    my_list = [(elem[1],elem[2],round(elem[3],2)) for elem in my_list]
    print(my_list)
    print()

=# Generating a regression model with the pca

In [ ]:
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker

def show_errors(y_real_train, y_pred_train, y_real_test, y_pred_test):
    """ plots the errors of a linear regression model
    :param y_real_train: target variable (price) for the train set
    :param y_pred_train: predicted price  from model on the train set
    :param y_real_test: target variable (price) for the test set
    :param y_pred_test: predicted price from model on the test set
    :return: array of plots comparing y_true and y_predict for test and train and distribution of residuals in both cases
    """

    plt.style.use('seaborn') 

    fig, ax = plt.subplots(2,2,figsize=(10,10))
    
    ax[0,0].scatter(x = np.exp(y_real_train), y = np.exp(y_pred_train), c = 'green')
    ax[0,0].plot([0,700000], [0,700000], linestyle = '--',c = 'black')
    ax[0,0].set_xlim(0,700000)
    ax[0,0].set_ylim(0,700000)

    ax[0,0].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[0,0].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[0,0].set_title('Train set')

    ax[0,1].hist(x = np.exp(y_real_train)-np.exp(y_pred_train), bins = 50,color = 'green')
    ax[0,1].set_xlim(-200000,200000)
    ax[0,1].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[0,1].set_title('Train set')

    ax[1,0].scatter(x = np.exp(y_real_test), y = np.exp(y_pred_test), c = 'blue')
    ax[1,0].plot([0,700000], [0,700000], linestyle = '--',c = 'black')
    ax[1,0].set_xlim(0,700000)
    ax[1,0].set_ylim(0,700000)
    ax[1,0].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[1,0].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[1,0].set_title('Test set')   

    ax[1,1].hist(x = np.exp(y_real_test)-np.exp(y_pred_test), bins = 50,color = 'blue')
    ax[1,1].set_xlim(-200000,200000)
    ax[1,1].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[1,1].set_title('Test set')

    fig.tight_layout()
    pass

In [ ]:
X_test

In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()

lm.fit(X_train,y_train)

y_pred_train = lm.predict(X_train)
y_pred_test  = lm.predict(X_test)

show_errors(y_train, y_pred_train, y_test, y_pred_test)

Error metrics

In [ ]:
lm.score(X_test,y_test), r2_score(y_test,y_pred_test)

In [ ]:
lm.score(X_train,y_train), r2_score(y_train,y_pred_train)